<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

***Some parts of the notebook are almost the copy of [ mmta-team course](https://github.com/mmta-team/mmta_fall_2020). Special thanks to mmta-team for making them publicly available. [Original notebook](https://github.com/mmta-team/mmta_fall_2020/blob/master/tasks/01_word_embeddings/task_word_embeddings.ipynb).***

<b> Прочитайте семинар, пожалуйста, для успешного выполнения домашнего задания. В конце ноутка напишите свой вывод. Работа без вывода оценивается ниже.

In [54]:
from functools import lru_cache

import pandas as pd

## Задача поиска схожих по смыслу предложений

Мы будем ранжировать вопросы [StackOverflow](https://stackoverflow.com) на основе семантического векторного представления 

До этого в курсе не было речи про задачу ранжировния, поэтому введем математическую формулировку

## Задача ранжирования(Learning to Rank)

* $X$ - множество объектов
* $X^l = \{x_1, x_2, ..., x_l\}$ - обучающая выборка
<br>На обучающей выборке задан порядок между некоторыми элементами, то есть нам известно, что некий объект выборки более релевантный для нас, чем другой:
* $i \prec j$ - порядок пары индексов объектов на выборке $X^l$ c индексами $i$ и $j$
### Задача:
построить ранжирующую функцию $a$ : $X \rightarrow R$ такую, что
$$i \prec j \Rightarrow a(x_i) < a(x_j)$$

<img src="https://d25skit2l41vkl.cloudfront.net/wp-content/uploads/2016/12/Featured-Image.jpg" width=500, height=450>

### Embeddings

Будем использовать предобученные векторные представления слов на постах Stack Overflow.<br>
[A word2vec model trained on Stack Overflow posts](https://github.com/vefstathiou/SO_word2vec)

In [55]:
# !wget https://zenodo.org/record/1199620/files/SO_vectors_200.bin?download=1

In [56]:
from gensim.models.keyedvectors import KeyedVectors

wv_embeddings = KeyedVectors.load_word2vec_format(
    fname="SO_vectors_200.bin", binary=True
)

#### Как пользоваться этими векторами?

Посмотрим на примере одного слова, что из себя представляет embedding

In [57]:
word = "dog"
if word in wv_embeddings:
    print(wv_embeddings[word].dtype, wv_embeddings[word].shape)

float32 (200,)


In [58]:
print(f"Num of words: {len(wv_embeddings.index_to_key)}")

Num of words: 1787145


Найдем наиболее близкие слова к слову `dog`:

#### Вопрос 1:
* Входит ли слово `cat` топ-5 близких слов к слову `dog`? Какое место? 

In [59]:
# method most_simmilar
dog_similar_dict = dict(wv_embeddings.most_similar(word))
similar_words = list(dog_similar_dict.keys())
print(dog_similar_dict)
print(
    f"Is 'cat' in the top-5 most similar words to 'dog': {'cat' in similar_words[:5]}."
)
print(
    f"Is 'cats' in the top-5 most similar words to 'dog': {'cats' in similar_words[:5]}, {similar_words.index('cats')}rd place."
)

{'animal': 0.8564180135726929, 'dogs': 0.7880866527557373, 'mammal': 0.7623804211616516, 'cats': 0.7621253728866577, 'animals': 0.760793924331665, 'feline': 0.7392398715019226, 'bird': 0.7315488457679749, 'animal1': 0.7219215631484985, 'doggy': 0.7213349342346191, 'labrador': 0.7209131717681885}
Is 'cat' in the top-5 most similar words to 'dog': False.
Is 'cats' in the top-5 most similar words to 'dog': True, 3rd place.


Не входит. Слово cats входит и стоит на 3 месте.

### Векторные представления текста

Перейдем от векторных представлений отдельных слов к векторным представлениям вопросов, как к **среднему** векторов всех слов в вопросе. Если для какого-то слова нет предобученного вектора, то его нужно пропустить. Если вопрос не содержит ни одного известного слова, то нужно вернуть нулевой вектор.

#### Выбор токенизатора

In [60]:
text = "I'm a good man, with six dogs and eight cats"

##### Regex

In [61]:
import re

import numpy as np


# you can use your tokenizer
# for example, from nltk.tokenize import WordPunctTokenizer
class MyTokenizer:
    def __init__(self):
        pass

    def tokenize(self, text):
        return re.findall("\w+", text)


regex_tokenizer = MyTokenizer()

In [62]:
regex_tokenizer.tokenize(text)

['I', 'm', 'a', 'good', 'man', 'with', 'six', 'dogs', 'and', 'eight', 'cats']

##### word_tokenize

In [63]:
# import nltk
# nltk.download('punkt')

In [64]:
from nltk.tokenize import word_tokenize

In [65]:
word_tokenize(text)

['I',
 "'m",
 'a',
 'good',
 'man',
 ',',
 'with',
 'six',
 'dogs',
 'and',
 'eight',
 'cats']

##### TweetTokenizer

In [66]:
from nltk.tokenize import TweetTokenizer

tweet_tokenizer = TweetTokenizer()

In [67]:
tweet_tokenizer.tokenize(text)

["I'm", 'a', 'good', 'man', ',', 'with', 'six', 'dogs', 'and', 'eight', 'cats']

##### TextBlob

In [68]:
# !pip install -U textblob
# !python3 -m textblob.download_corpora

In [69]:
from textblob import TextBlob

blob = TextBlob(text)
blob.words

WordList(['I', "'m", 'a', 'good', 'man', 'with', 'six', 'dogs', 'and', 'eight', 'cats'])

We could notice that the TextBlob tokenizer removes the punctuation.  
In addition, it doesn't have  rules for English contractions.

##### Spacy

In [70]:
# !pip install spacy
# !python3 -m spacy download en_core_web_sm

In [71]:
from spacy.lang.en import English
from spacy.tokenizer import Tokenizer

class SpacyTokenizer:
    def __init__(self):
        nlp = English()
        self.tokenizer = nlp.tokenizer

    def tokenize(self, text):
        return [str(token) for token in self.tokenizer(text)]

In [72]:
spacy_tokenizer = SpacyTokenizer()
spacy_tokenizer.tokenize(text)

['I',
 "'m",
 'a',
 'good',
 'man',
 ',',
 'with',
 'six',
 'dogs',
 'and',
 'eight',
 'cats']

Выберем пока что TweetTokenizer

In [73]:
tokenizer = TweetTokenizer()

In [74]:
def question_to_vec(question, embeddings, tokenizer, dim=200, pretrained=True):
    """
    question: строка
    embeddings: наше векторное представление
    dim: размер любого вектора в нашем представлении

    return: векторное представление для вопроса
    """
    if not pretrained:
        embeddings = embeddings.wv

    vector = np.zeros((dim,))
    vectors = []
    for word in tokenizer.tokenize(question):
        if word in embeddings.key_to_index:
            vectors.append(embeddings[word])
        else:
            continue

    if vectors:
        vector = np.mean(vectors, axis=0)

    return vector

Теперь у нас есть метод для создания векторного представления любого предложения.

#### Вопрос 2:
* Какая третья(с индексом 2) компонента вектора предложения `I love neural networks` (округлите до 2 знаков после запятой)?

In [75]:
v = question_to_vec("I love neural networks", wv_embeddings, tokenizer)
v[2].round(2)

-1.29

### Оценка близости текстов

Представим, что мы используем идеальные векторные представления слов. Тогда косинусное расстояние между дублирующими предложениями должно быть меньше, чем между случайно взятыми предложениями. 

Сгенерируем для каждого из $N$ вопросов $R$ случайных отрицательных примеров и примешаем к ним также настоящие дубликаты. Для каждого вопроса будем ранжировать с помощью нашей модели $R + 1$ примеров и смотреть на позицию дубликата. Мы хотим, чтобы дубликат был первым в ранжированном списке.

#### Hits@K
Первой простой метрикой будет количество корректных попаданий для какого-то $K$:
$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [rank\_q_i^{'} \le K],$$
* $\begin{equation*}
[x < 0 ] \equiv 
 \begin{cases}
   1, &x < 0\\
   0, &x \geq 0
 \end{cases}
\end{equation*}$ - индикаторная функция
* $q_i$ - $i$-ый вопрос
* $q_i^{'}$ - его дубликат
* $rank\_q_i^{'}$ - позиция дубликата в ранжированном списке ближайших предложений для вопроса $q_i$.

#### DCG@K
Второй метрикой будет упрощенная DCG метрика, учитывающая порядок элементов в списке путем домножения релевантности элемента на вес равный обратному логарифму номера позиции:
$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank\_q_i^{'})}\cdot[rank\_q_i^{'} \le K],$$
С такой метрикой модель штрафуется за большой ранк корректного ответа

#### Вопрос 3:
* Максимум `Hits@47 - DCG@1`?

Максимум этого выражения 1, если $\text{Hits}@47$ равен $1$ и $\text{DCG}@1$ равен $0$.

<img src='https://hsto.org/files/1c5/edf/dee/1c5edfdeebce4b71a86bdf986d9f88f2.jpg' width=400, height=200>

#### Пример оценок

Вычислим описанные выше метрики для игрушечного примера. 
Пусть
* $N = 1$, $R = 3$
* <font color='green'>"Что такое python?"</font> - вопрос $q_1$
* <font color='red'>"Что такое язык python?"</font> - его дубликат $q_i^{'}$

Пусть модель выдала следующий ранжированный список кандидатов:

1. "Как изучить с++?"
2. <font color='red'>"Что такое язык python?"</font>
3. "Хочу учить Java"
4. "Не понимаю Tensorflow"

$\Rightarrow rank\_q_i^{'} = 2$

Вычислим метрику *Hits@K* для *K = 1, 4*:

- [K = 1] $\text{Hits@1} =  [rank\_q_i^{'} \le 1)] = 0$
- [K = 4] $\text{Hits@4} =  [rank\_q_i^{'} \le 4] = 1$

Вычислим метрику *DCG@K* для *K = 1, 4*:
- [K = 1] $\text{DCG@1} = \frac{1}{\log_2(1+2)}\cdot[2 \le 1] = 0$
- [K = 4] $\text{DCG@4} = \frac{1}{\log_2(1+2)}\cdot[2 \le 4] = \frac{1}{\log_2{3}}$

#### Вопрос 4:
* Вычислите `DCG@10`, если $rank\_q_i^{'} = 9$(округлите до одного знака после запятой)

$DCG@10 = \frac {1}{log_2(1+9)}\cdot[9 \leq 10] = \frac {1}{log_2(10)} = 0.3$

In [76]:
(1 / np.log2(10)).round(1)

0.3

### HITS\_COUNT и DCG\_SCORE

Каждая функция имеет два аргумента: $dup\_ranks$ и $k$. $dup\_ranks$ является списком, который содержит рейтинги дубликатов(их позиции в ранжированном списке). Например, $dup\_ranks = [2]$ для примера, описанного выше.

In [77]:
def hits_count(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть  Hits@k
    """
    hits = [1 if rank <= k else 0 for rank in dup_ranks]

    return np.mean(hits)

In [78]:
def dcg_score(dup_ranks, k):
    """
    dup_ranks: list индексов дубликатов
    result: вернуть DCG@k
    """
    dcgs = [1 / np.log2(1 + rank) if rank <= k else 0 for rank in dup_ranks]

    return np.mean(dcgs)

Протестируем функции. Пусть $N = 1$, то есть один эксперимент. Будем искать копию вопроса и оценивать метрики.

In [79]:
copy_answers = [
    "How does the catch keyword determine the type of exception that was thrown",
]

# наши кандидаты
candidates_ranking = [
    [
        "How Can I Make These Links Rotate in PHP",
        "How does the catch keyword determine the type of exception that was thrown",
        "NSLog array description not memory address",
        "PECL_HTTP not recognised php ubuntu",
    ],
]
# dup_ranks — позиции наших копий, так как эксперимент один, то этот массив длины 1
dup_ranks = [2]

# вычисляем метрику для разных k
print("Ваш ответ HIT:", [hits_count(dup_ranks, k) for k in range(1, 5)])
print("Ваш ответ DCG:", [round(dcg_score(dup_ranks, k), 5) for k in range(1, 5)])

Ваш ответ HIT: [0.0, 1.0, 1.0, 1.0]
Ваш ответ DCG: [0.0, 0.63093, 0.63093, 0.63093]


У вас должно получиться

In [80]:
# correct_answers - метрика для разных k
correct_answers = pd.DataFrame(
    [[0, 1, 1, 1], [0, 1 / (np.log2(3)), 1 / (np.log2(3)), 1 / (np.log2(3))]],
    index=["HITS", "DCG"],
    columns=range(1, 5),
)
correct_answers

,1,2,3,4
HITS,0,1.00000,1.00000,1.00000
DCG,0,0.63093,0.63093,0.63093


### Данные
[arxiv link](https://drive.google.com/file/d/1QqT4D0EoqJTy7v9VrNCYD-m964XZFR7_/edit)

`train.tsv` - выборка для обучения.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>**

`validation.tsv` - тестовая выборка.<br> В каждой строке через табуляцию записаны: **<вопрос>, <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...**

In [81]:
# !unzip stackoverflow_similar_questions.zip

In [82]:
import re

Считайте данные.

In [83]:
def read_corpus(filename):
    data = []
    for line in open(filename, encoding="utf-8"):
        data.append(re.split(r"\t+", line.rstrip("\t").rstrip("\n")))
    return data

Нам понадобиться только файл validation.

In [84]:
validation_data = read_corpus("./stackoverflow_similar_questions/data/validation.tsv")

Кол-во строк

In [85]:
len(validation_data)

3760

Размер нескольких первых строк

In [86]:
for i in range(5):
    print(i + 1, len(validation_data[i]))

1 1001
2 1001
3 1001
4 1001
5 1001


### Ранжирование без обучения

Реализуйте функцию ранжирования кандидатов на основе косинусного расстояния. Функция должна по списку кандидатов вернуть отсортированный список пар (позиция в исходном списке кандидатов, кандидат). При этом позиция кандидата в полученном списке является его рейтингом (первый - лучший). Например, если исходный список кандидатов был [a, b, c], и самый похожий на исходный вопрос среди них - c, затем a, и в конце b, то функция должна вернуть список **[(2, c), (0, a), (1, b)]**.

In [87]:
from copy import deepcopy

from sklearn.metrics.pairwise import cosine_similarity

In [88]:
def rank_candidates(
    question, candidates, embeddings, tokenizer, dim=200, pretrained=True
):
    """
    question: строка
    candidates: массив строк(кандидатов) [a, b, c]
    result: пары (начальная позиция, кандидат) [(2, c), (0, a), (1, b)]
    """

    if not pretrained:
        embeddings = embeddings.wv

    question_emb = np.array(
        question_to_vec(question, embeddings, tokenizer, dim)
    ).reshape(1, -1)

    total_resuls = []

    for i, canditate in enumerate(candidates):
        candidate_emb = np.array(
            question_to_vec(canditate, embeddings, tokenizer, dim)
        ).reshape(1, -1)

        total_resuls.append(
            (
                i,
                canditate,
                cosine_similarity(question_emb, candidate_emb)[0][0],
            )
        )
    total_resuls_sorted = sorted(total_resuls, key=lambda item: item[2], reverse=True)

    result = [(tuple_[0], tuple_[1]) for tuple_ in total_resuls_sorted]

    return result

Протестируйте работу функции на примерах ниже. Пусть $N=2$, то есть два эксперимента

In [89]:
questions = ["converting string to list", "Sending array via Ajax fails"]

candidates = [
    [
        "Convert Google results object (pure js) to Python object",  # первый эксперимент
        "C# create cookie from string and send it",
        "How to use jQuery AJAX for an outside domain?",
    ],
    [
        "Getting all list items of an unordered list in PHP",  # второй эксперимент
        "WPF- How to update the changes in list item of a list",
        "select2 not displaying search results",
    ],
]

In [90]:
for question, q_candidates in zip(questions, candidates):
    ranks = rank_candidates(question, q_candidates, wv_embeddings, tokenizer)
    print(ranks)
    print()

[(1, 'C# create cookie from string and send it'), (0, 'Convert Google results object (pure js) to Python object'), (2, 'How to use jQuery AJAX for an outside domain?')]

[(1, 'WPF- How to update the changes in list item of a list'), (0, 'Getting all list items of an unordered list in PHP'), (2, 'select2 not displaying search results')]



Для первого экперимента вы можете полностью сравнить ваши ответы и правильные ответы. Но для второго эксперимента два ответа на кандидаты будут <b>скрыты</b>(*)

In [91]:
# должно вывести
results = [
    [
        (1, "C# create cookie from string and send it"),
        (0, "Convert Google results object (pure js) to Python object"),
        (2, "How to use jQuery AJAX for an outside domain?"),
    ],
    [
        ("*", "Getting all list items of an unordered list in PHP"),  # скрыт
        ("*", "select2 not displaying search results"),  # скрыт
        ("*", "WPF- How to update the changes in list item of a list"),
    ],
]  # скрыт
results

[[(1, 'C# create cookie from string and send it'),
  (0, 'Convert Google results object (pure js) to Python object'),
  (2, 'How to use jQuery AJAX for an outside domain?')],
 [('*', 'Getting all list items of an unordered list in PHP'),
  ('*', 'select2 not displaying search results'),
  ('*', 'WPF- How to update the changes in list item of a list')]]

Последовательность начальных индексов вы должны получить `для эксперимента 1`  1, 0, 2.

#### Вопрос 5:
* Какую последовательность начальных индексов вы получили `для эксперимента 2`(перечисление без запятой и пробелов, например, `102` для первого эксперимента?

102

Теперь мы можем оценить качество нашего метода. Запустите следующие два блока кода для получения результата. Обратите внимание, что вычисление расстояния между векторами занимает некоторое время (примерно 10 минут). Можете взять для validation 1000 примеров.

In [92]:
from tqdm.notebook import tqdm

In [93]:
def get_ranking(
    validation_data,
    emb,
    tokenizer,
    max_validation_examples=100,
    dim=200,
    pretrained=True,
):
    wv_ranking = []
    for i, line in enumerate(tqdm(validation_data)):
        if i == max_validation_examples:
            break
        q, *ex = line
        ranks = rank_candidates(q, ex, emb, tokenizer, dim, pretrained)
        wv_ranking.append([r[0] for r in ranks].index(0) + 1)
    return wv_ranking

In [94]:
wv_ranking_base_tokenizer = get_ranking(validation_data, wv_embeddings, regex_tokenizer)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (
            k,
            dcg_score(wv_ranking_base_tokenizer, k),
            k,
            hits_count(wv_ranking_base_tokenizer, k),
        )
    )

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.260 | Hits@   1: 0.260
DCG@   5: 0.327 | Hits@   5: 0.390
DCG@  10: 0.343 | Hits@  10: 0.440
DCG@ 100: 0.391 | Hits@ 100: 0.670
DCG@ 500: 0.419 | Hits@ 500: 0.890
DCG@1000: 0.431 | Hits@1000: 1.000


### Эмбеддинги, обученные на корпусе похожих вопросов

In [95]:
train_data = read_corpus("./stackoverflow_similar_questions/data/train.tsv")

Улучшите качество модели.<br>Склеим вопросы в пары и обучим на них модель Word2Vec из gensim. Выберите размер window. Объясните свой выбор.

In [96]:
train_corpus = []
for pair in train_data:
    train_corpus.extend(pair)

In [97]:
from gensim import utils


class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        for sentence in train_corpus:
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(sentence)

In [98]:
sentences = MyCorpus()

In [99]:
from gensim.models import Word2Vec

embeddings_trained = Word2Vec(
    sentences,  # data for model to train on
    vector_size=200,  # embedding vector size
    min_count=5,  # consider words that occured at least 5 times
    window=5,
    workers=8,
    sg=1,
)

In [100]:
wv_ranking_from_scratch = []
max_validation_examples = 100
for i, line in enumerate(tqdm(validation_data)):
    if i == max_validation_examples:
        break
    q, *ex = line
    ranks = rank_candidates(q, ex, embeddings_trained, tokenizer, pretrained=False)
    wv_ranking_from_scratch.append([r[0] for r in ranks].index(0) + 1)

  0%|          | 0/3760 [00:00<?, ?it/s]

In [101]:
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (
            k,
            dcg_score(wv_ranking_from_scratch, k),
            k,
            hits_count(wv_ranking_from_scratch, k),
        )
    )

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.250 | Hits@   1: 0.250
DCG@   5: 0.296 | Hits@   5: 0.340
DCG@  10: 0.305 | Hits@  10: 0.370
DCG@ 100: 0.354 | Hits@ 100: 0.610
DCG@ 500: 0.385 | Hits@ 500: 0.850
DCG@1000: 0.400 | Hits@1000: 1.000


### Замечание:
Решить эту задачу с помощью обучения полноценной нейронной сети будет вам предложено, как часть задания в одной из домашних работ по теме "Диалоговые системы".

### Пробуем улучшить результаты

#### Поработаем с предобученными эмбеддингами

##### Tokenizers

* regex_tokenizer - базовый токенизатор
* tweet_tokenizer - TweetTokenizer из NLTK 
* spacy_tokenizer

In [102]:
wv_ranking_base_tokenizer = get_ranking(validation_data, wv_embeddings, regex_tokenizer)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (
            k,
            dcg_score(wv_ranking_base_tokenizer, k),
            k,
            hits_count(wv_ranking_base_tokenizer, k),
        )
    )

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.260 | Hits@   1: 0.260
DCG@   5: 0.327 | Hits@   5: 0.390
DCG@  10: 0.343 | Hits@  10: 0.440
DCG@ 100: 0.391 | Hits@ 100: 0.670
DCG@ 500: 0.419 | Hits@ 500: 0.890
DCG@1000: 0.431 | Hits@1000: 1.000


In [103]:
wv_ranking_tweet_tokenizer = get_ranking(
    validation_data, wv_embeddings, tweet_tokenizer
)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (
            k,
            dcg_score(wv_ranking_tweet_tokenizer, k),
            k,
            hits_count(wv_ranking_tweet_tokenizer, k),
        )
    )

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.270 | Hits@   1: 0.270
DCG@   5: 0.341 | Hits@   5: 0.410
DCG@  10: 0.350 | Hits@  10: 0.440
DCG@ 100: 0.399 | Hits@ 100: 0.680
DCG@ 500: 0.425 | Hits@ 500: 0.890
DCG@1000: 0.436 | Hits@1000: 1.000


In [105]:
wv_ranking_spacy_tokenizer = get_ranking(
    validation_data, wv_embeddings, spacy_tokenizer
)
for k in tqdm([1, 5, 10, 100, 500, 1000]):
    print(
        "DCG@%4d: %.3f | Hits@%4d: %.3f"
        % (
            k,
            dcg_score(wv_ranking_spacy_tokenizer, k),
            k,
            hits_count(wv_ranking_spacy_tokenizer, k),
        )
    )

  0%|          | 0/3760 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

DCG@   1: 0.240 | Hits@   1: 0.240
DCG@   5: 0.306 | Hits@   5: 0.370
DCG@  10: 0.325 | Hits@  10: 0.430
DCG@ 100: 0.372 | Hits@ 100: 0.660
DCG@ 500: 0.401 | Hits@ 500: 0.890
DCG@1000: 0.413 | Hits@1000: 1.000


TweetTokenizer лучше всех - берем его для дальнейших исследований

Напишите свой вывод о полученных результатах.
* Какой принцип токенизации даёт качество лучше и почему?  
`TweetTokenizer лучше всех, потому что он качественно отделяет слова`
* Помогает ли нормализация слов?
* Какие эмбеддинги лучше справляются с задачей и почему?
* Почему получилось плохое качество решения задачи?
* Предложите свой подход к решению задачи.

## Вывод:


Токенизаторы: 
